In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
news_df = pd.read_csv('data.csv')

In [5]:
news_df.head()

,id,label,date,title,summary,source
0,83,1,2023-03-04 06:18:13+00,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",Anggota Komisi VII DPR RI Rofik Hananto menyay...,tempo
1,84,0,2023-03-04 06:04:38+00,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,Presiden Joko Widodo telah memerintahkan Wakil...,tempo
2,85,0,2023-03-04 06:18:04+00,HNW Mendukung Jamaah Umroh First Travel Dapatk...,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...,tempo
3,86,0,2023-03-04 06:44:10+00,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...,tempo
4,87,0,2023-03-04 06:38:57+00,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,Ketua MPR RI Bambang Soesatyo telah diangkat s...,tempo


In [6]:
news_df.shape

(22009, 6)

In [7]:
news_df.isna().sum()

id          0
label       0
date        0
title       0
summary    10
source      0
dtype: int64

In [8]:
news_df = news_df.fillna(' ')

In [9]:
news_df.isna().sum()

id         0
label      0
date       0
title      0
summary    0
source     0
dtype: int64

In [10]:
news_df['konten'] = news_df['title']+" "+news_df['summary']

In [11]:
news_df

,id,label,date,title,summary,source,konten
0,83,1,2023-03-04 06:18:13+00,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",Anggota Komisi VII DPR RI Rofik Hananto menyay...,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert..."
1,84,0,2023-03-04 06:04:38+00,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,Presiden Joko Widodo telah memerintahkan Wakil...,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...
2,85,0,2023-03-04 06:18:04+00,HNW Mendukung Jamaah Umroh First Travel Dapatk...,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...
3,86,0,2023-03-04 06:44:10+00,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...
4,87,0,2023-03-04 06:38:57+00,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,Ketua MPR RI Bambang Soesatyo telah diangkat s...,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...
...,...,...,...,...,...,...,...
22004,44725,0,2023-04-02 05:46:00+00,Indonesia Batal Jadi Tuan Rumah Piala Dunia U-...,Jokowi mengaku pusing dengan urusan Piala Duni...,tempo,Indonesia Batal Jadi Tuan Rumah Piala Dunia U-...
22005,44726,0,2023-04-02 05:58:27+00,Rentetan Kebakaran Pertamina 2023: Depo Plumpa...,Sejumlah kebakaran terjadi pada 2023 yang terk...,tempo,Rentetan Kebakaran Pertamina 2023: Depo Plumpa...
22006,44733,0,2023-04-02 06:00:00+00,Update Ledakan di Kilang Dumai: Pertamina Foku...,PT Kilang Pertamina Internasional (PT KPI) Ref...,cnbcindonesia,Update Ledakan di Kilang Dumai: Pertamina Foku...
22007,44745,0,2023-04-02 06:09:55+00,Rilis Lagu Marhaban Yaa Ramadhan Ayu Azhari Li...,"Ayu Ting-Ting merilis single religi ""Marhaban ...",kumparan,Rilis Lagu Marhaban Yaa Ramadhan Ayu Azhari Li...


In [12]:
news_df['konten']

0        Depo Plumpang Terbakar, Anggota DPR Minta Pert...
1        Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...
2        HNW Mendukung Jamaah Umroh First Travel Dapatk...
3        Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...
4        Bamsoet Ajak Komunitas Otomotif Kembangkan Per...
                               ...                        
22004    Indonesia Batal Jadi Tuan Rumah Piala Dunia U-...
22005    Rentetan Kebakaran Pertamina 2023: Depo Plumpa...
22006    Update Ledakan di Kilang Dumai: Pertamina Foku...
22007    Rilis Lagu Marhaban Yaa Ramadhan Ayu Azhari Li...
22008    Polisi Bakal Ekshumasi Jenazah Istri yang Dira...
Name: konten, Length: 22009, dtype: object

In [13]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('indonesian')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news_df['konten'] = news_df['konten'].apply(stemming) 

In [ ]:
news_df['konten']

0        depo plumpang terbakar anggota dpr pertamina p...
1        jokowi perintahkan wapr ma ruf amin tinjau lok...
2        hnw mendukung jamaah umroh first travel dapatk...
3        tim dokk polri terima kantong jenazah korban k...
4        bamsoet ajak komunita otomotif kembangkan pere...
                               ...                        
22004    indonesia batal tuan rumah piala dunia u jokow...
22005    rentetan kebakaran pertamina depo plumpang kap...
22006    updat ledakan kilang dumai pertamina foku pemu...
22007    rili lagu marhaban yaa ramadhan ayu azhari lib...
22008    polisi ekshumasi jenazah istri diracun suaminy...
Name: konten, Length: 22009, dtype: object

In [ ]:
X = news_df['konten'].values
y = news_df['label'].values

In [ ]:
print(X)

['depo plumpang terbakar anggota dpr pertamina pastikan pasokan bbm terganggu anggota komisi vii dpr ri rofik hananto menyayangkan insiden kebakaran depo plumpang jakarta utara jumat maret rofik pertamina pasokan bahan bakar minyak aman insiden depo plumpang merupak'
 'jokowi perintahkan wapr ma ruf amin tinjau lokasi kebakaran depo plumpang presiden joko widodo memerintahkan wakil presiden ma ruf amin meninjau langsung lokasi kebakaran depo pertamina plumpang jakarta utara kebakaran lokasi jumat malam maret mengakibatkan belasan warga dep'
 'hnw mendukung jamaah umroh first travel dapatkan haknya wakil ketua mpr ri dr h m hidayat nur wahid ma hnw menerima kunjungan perwakilan korban biro perjalanan haji umroh first travel tergabung paguyuban first travel indonesia pertemuan ruang kerja wakil ketua mpr gedung n'
 ...
 'updat ledakan kilang dumai pertamina foku pemulihan pt kilang pertamina internasion pt kpi refineri unit ru dumai beroperasi kejadian area ga compressor beralih prose re

In [ ]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [ ]:
print(X)

  (0, 36612)	0.16055543294875263
  (0, 36432)	0.09841040439707406
  (0, 34595)	0.14521556859541587
  (0, 34464)	0.12166255163766222
  (0, 29892)	0.4355591515017619
  (0, 29666)	0.07653943195492247
  (0, 27697)	0.3709626409444876
  (0, 27164)	0.20838780949538377
  (0, 24990)	0.12626631539787259
  (0, 24976)	0.24468851979230327
  (0, 22402)	0.1114118282277803
  (0, 22112)	0.21777957575088094
  (0, 21738)	0.1432636165746843
  (0, 19111)	0.07288371318120136
  (0, 16687)	0.0951105527350968
  (0, 14972)	0.10864184632698183
  (0, 14240)	0.10834210905547727
  (0, 13665)	0.06875679913487812
  (0, 13142)	0.24538708522871255
  (0, 11686)	0.21777957575088094
  (0, 8883)	0.17274713927707358
  (0, 6531)	0.368433795163236
  (0, 2658)	0.11412560940609509
  (0, 2189)	0.12427759582605097
  (0, 2131)	0.10540172624028797
  :	:
  (22008, 25544)	0.18242522877827838
  (22008, 25233)	0.1400463856433975
  (22008, 24546)	0.10847637778027383
  (22008, 22361)	0.09452250288570482
  (22008, 21921)	0.189896166612704

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
X_train.shape

(17607, 37829)

In [ ]:
X_test.shape

(4402, 37829)

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.7753166354290907


In [ ]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.5327124034529759


In [ ]:
# prediction system

input_data = X_test[10].reshape(1, -1)
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Real news


In [ ]:
news_df['konten'][10]

'erick thohir heru dirut pertamina langsung baha nasib depo plumpang diperintah jokowi kebakaran depo pertamina presiden jokowi memerintahkan pertamina gubernur dki jakarta menteri bumn mencari solusi opsi depo dipindah warga tinggal direlokasi wakil presiden ma ruf amin memilih dipindahkan pelabuhan jokowi menyebut depo pertamina plumpang zona bahaya zona air'